# Finding the shortest path in an amusement park

## Section 1 : Setup 

In [1]:
# import 
import cv2
import tkinter as tk
from tkinter.simpledialog import askstring

In [2]:
# Function to update the image 
#--------------------------------------------------------------------------------------------------------

def update_display():

    '''
    The aim of this is to update the image with the points added during a loop (in red). 

    Return : 
    None

    '''

    #--------------------------------------------------------------------------------------------------------
    
    global actual_image # The actual image with the previously added points (in blue)
    global points # The points to add in the loop (in red) 
    global current_zoom, x_offset, y_offset # The zoom parameters
    
    #--------------------------------------------------------------------------------------------------------

    # We zoom on the original image 
    zoomed_image = cv2.resize(actual_image.copy(), None, fx=current_zoom, fy=current_zoom, interpolation=cv2.INTER_LINEAR)
    image = zoomed_image.copy()

    #--------------------------------------------------------------------------------------------------------
    
    # Calculate the maximum allowable offsets to avoid going out of bounds (the program crahses and return an error in this case)
    max_x_offset = max(image.shape[1] - actual_image.shape[1], 0)
    max_y_offset = max(image.shape[0] - actual_image.shape[0], 0)

    #--------------------------------------------------------------------------------------------------------
    
    # Limit x_offset and y_offset to stay within bounds
    x_offset = max(0, min(x_offset, max_x_offset))
    y_offset = max(0, min(y_offset, max_y_offset))

    #--------------------------------------------------------------------------------------------------------
    
    # Apply offsets to simulate movement
    image = image[y_offset:y_offset + actual_image.shape[0], x_offset:x_offset + actual_image.shape[1]]

    #--------------------------------------------------------------------------------------------------------
    
    # Draw the points on the zoomed image at their original positions
    for point_name, point in points.items():

        x, y = point

        zoomed_x = int((x - x_offset) * current_zoom)
        zoomed_y = int((y - y_offset) * current_zoom)

        # Add a blue circle on the position
        cv2.circle(image, (zoomed_x, zoomed_y), 5, (0, 0, 255), -1)
    
    cv2.imshow('Image with Points', image)

In [3]:
# Function to add points
#--------------------------------------------------------------------------------------------------------

def add_point(event, x, y, flags, param):

    '''
    The aim of this function is to add points on the loaded image. The added point will be a node and should be an interest point on the map.

    Arg :
    event : An action defined in cv2. Here it is the right click to add a point. 
    x,y : The position of the point in the current window (as we allow zoom).
    flags,param : not used, mandatory to use this function

    Return : 
    None
    '''

    global points # The points to add in the loop (in red) 
    global all_points # The already existing points (in blue)
    global x_offset, y_offset, current_zoom # The zoom parameters

    # When the right click is pressed 
    if event == cv2.EVENT_RBUTTONDOWN:

        #--------------------------------------------------------------------------------------------------------

        # Store the point in the original image coordinates (as we'rz working on a zommed window)
        original_x = int((x + x_offset) / current_zoom)
        original_y = int((y + y_offset) / current_zoom)

        #--------------------------------------------------------------------------------------------------------
        
        # Create a Tkinter root window (hidden). Mean window of the program
        root = tk.Tk()
        root.withdraw()

        #--------------------------------------------------------------------------------------------------------
        
        # Display an input dialog to enter the name for the point
        point_name = askstring("Point Name", "Enter a name for the point:")

        #--------------------------------------------------------------------------------------------------------
        
        # Check if a name was entered
        if point_name:

            # Store the point in the dictionaries of the names
            points[point_name] = (original_x,original_y)
            all_points[point_name] = (original_x,original_y)

        #--------------------------------------------------------------------------------------------------------

        # Destroy the root window
        root.destroy()

        #--------------------------------------------------------------------------------------------------------

        # Update the image using our function defined above
        update_display()

In [4]:
# Mean code to add point 
#------------------------------------------------------------------------------------------------------------------

# We define a dict that will contain all ever added points
all_points = {}

#------------------------------------------------------------------------------------------------------------------

# We load the original image and make a copy of it on which we'll add points 
original_image = cv2.imread('Map.JPG')

# Create a copy of the original that we will update 
image_copy = original_image.copy()
cv2.imwrite('Map_with_points.JPG', image_copy)

#------------------------------------------------------------------------------------------------------------------

boo = True
while boo :  

    # We define a dictionnary to contain all added points during the actual loop !!!
    # The mean idea is that adding all points at once is heavy and so, you can do it step by step with 2 commands (see later)
    points = {}

    #------------------------------------------------------------------------------------------------------------------

    # Zoom parameters
    current_zoom = 1.0

    # Movement parameters
    x_offset = 0
    y_offset = 0

    #------------------------------------------------------------------------------------------------------------------

    # Load your JPEG image
    actual_image = cv2.imread('Map_with_points.JPG')
    image = actual_image.copy()

    #------------------------------------------------------------------------------------------------------------------

    # Create a window and set the mouse callback function
    cv2.namedWindow('Image with Points')
    cv2.setMouseCallback('Image with Points', add_point)

    #------------------------------------------------------------------------------------------------------------------
    
    # Displays what has been done to this point 
    cv2.imshow('Image with Points', image)

    while True:        

        #------------------------------------------------------------------------------------------------------------------

        # Wait for user to add points and press a key
        key = cv2.waitKey(1) & 0xFF

        if key == ord('q'): # You finished part of the points
            break

        elif key == ord('y'): # You finished adding all your points
            boo = False 
            break 

    #------------------------------------------------------------------------------------------------------------------
    # Save the final image with points

    final_image_with_points = original_image.copy()

    for point_name, point in all_points.items():

        x, y = point

        # We add a blue circle at pointed points
        cv2.circle(final_image_with_points, (x, y), 5, (255, 0, 0), -1)

    cv2.imwrite('Map_with_points.JPG', final_image_with_points)

    #------------------------------------------------------------------------------------------------------------------

    cv2.destroyAllWindows()

#------------------------------------------------------------------------------------------------------------------

# Print all points with their names
for name, location in all_points.items():
    print(f"Name: {name}, Location: {location}")

#------------------------------------------------------------------------------------------------------------------
# And write them in an external file 

# Specify the name of the output text file
output_file = "points.txt"

# Open the file in write mode
with open(output_file, "w") as file:
    for name, location in all_points.items():
        file.write(str(name) + ';' + str(location) + '\n')

Name: pandas, Location: (284, 284)
Name: lion, Location: (389, 172)
Name: tiger, Location: (788, 219)
Name: bear, Location: (918, 486)
